# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_city = pd.read_csv("city_weather.csv")
weather_city_df = pd.DataFrame(weather_city)
weather_city_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.80,-68.30,37.40,74,75,25.28,AR,1595600382
1,Rikitea,-23.12,-134.97,67.77,57,0,12.66,PF,1595600384
2,Saint Paul Harbor,57.79,-152.41,55.40,87,90,0.87,US,1595600511
3,Arraial do Cabo,-22.97,-42.03,77.00,61,0,19.46,BR,1595600404
4,East London,-33.02,27.91,73.40,43,0,8.05,ZA,1595600512


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Store latitude and longitude in locations
locations = weather_city_df[["Latitude", "Longitude"]]

# Convert humidity to float
humidity = weather_city_df["Humidity"].astype(float)

In [4]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=16,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_city_df.loc[(weather_city_df["Temperature"] < 80) & (weather_city_df["Temperature"] > 70)\
                                      & (weather_city_df["Wind Speed"] < 10) & (weather_city_df["Cloudiness"] == 0)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
4,East London,-33.02,27.91,73.40,43,0,8.05,ZA,1595600512
56,Castro,-24.79,-50.01,70.75,51,0,9.10,BR,1595600392
78,Srednekolymsk,67.45,153.68,71.02,40,0,7.16,RU,1595600426
113,Port Alfred,-33.59,26.89,75.27,35,0,7.49,ZA,1595600532
182,Port Elizabeth,-33.92,25.57,77.00,41,0,2.24,ZA,1595600432
184,Karratha,-20.74,116.85,70.68,49,0,2.91,AU,1595600547
193,Ust-Tsilma,65.44,52.15,79.34,31,0,8.25,RU,1595600551
213,Ubatuba,-23.43,-45.07,71.60,60,0,3.36,BR,1595600555
236,Binga,-17.62,27.34,72.97,33,0,7.81,ZW,1595600563
316,Acheng,45.55,126.98,75.20,94,0,2.24,CN,1595600579


In [6]:
# Hotel Map

# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
4,East London,ZA,-33.02,27.91,
56,Castro,BR,-24.79,-50.01,
78,Srednekolymsk,RU,67.45,153.68,
113,Port Alfred,ZA,-33.59,26.89,
182,Port Elizabeth,ZA,-33.92,25.57,
184,Karratha,AU,-20.74,116.85,
193,Ust-Tsilma,RU,65.44,52.15,
213,Ubatuba,BR,-23.43,-45.07,
236,Binga,ZW,-17.62,27.34,
316,Acheng,CN,45.55,126.98,


In [8]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
4,East London,ZA,-33.02,27.91,Tu Casa
56,Castro,BR,-24.79,-50.01,CHACARA BAILLY
78,Srednekolymsk,RU,67.45,153.68,
113,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
182,Port Elizabeth,ZA,-33.92,25.57,39 On Nile Guest House
184,Karratha,AU,-20.74,116.85,ibis Styles Karratha
193,Ust-Tsilma,RU,65.44,52.15,U Praskov'i
213,Ubatuba,BR,-23.43,-45.07,Hotel Pousada Dellamares
236,Binga,ZW,-17.62,27.34,Masumu River Lodge
316,Acheng,CN,45.55,126.98,Home Inn Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(hotel_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))